In [2]:
import requests as re
from bs4 import BeautifulSoup as bs
import pandas as pd
import sqlite3 as lite

In [3]:
url_initial = "https://pt.wikisource.org/wiki/Categoria:Literatura_brasileira"

In [4]:
response = re.get(url_initial)
soup = bs(response.text, 'html5lib')
links = soup.find('div', class_='mw-category').find_all('a')

In [5]:
def f_links(url):
    response = re.get(url)
    soup = bs(response.text, 'html5lib')
    #links = soup.find('div', class_='content').find_all('a')
    links = soup.find_all('a')
    return links

In [92]:
links = d['profundidade_2']

In [23]:
def buscar(chave, profundidade=1):
    dic = {'profundidade_%s'%i: [] for i in range(profundidade+1)}
    dic['profundidade_0'] = links
    for key in range(1, profundidade+1):
        for a in dic['profundidade_%s'%(key-1)]:
            try:
                if a['href'][:5] == '/wiki':
                    link = 'https://pt.m.wikisource.org' + a['href']
                elif 'https://pt' in a['href']:
                    link = a['href']
                    dic['profundidade_%s'%(key)].extend(f_links(link))
            except KeyError:
                pass
    return dic

In [93]:
b = buscar('qualquer coisa', 1)

In [96]:
len(b['profundidade_1'])

825708

In [28]:
len(a['profundidade_1'])#.keys()

31100

In [60]:
d['profundidade_2'] = a['profundidade_1']

31100

In [97]:
lista = []
lista.extend(d['profundidade_0'])
lista.extend(d['profundidade_1'])
lista.extend(d['profundidade_2'])
lista.extend(b['profundidade_1'])

In [98]:
dic_dt = {'titulo': [], 'link': []}

In [99]:
for tag in lista:
    try:
        if 'https://' in tag['href']:
            dic_dt['titulo'].append(tag['title'])
            dic_dt['link'].append(tag['href'])
        elif tag['href'][:5] == '/wiki':
            dic_dt['titulo'].append(tag['title'])
            dic_dt['link'].append('https://pt.m.wikisource.org' + tag['href'])
    except:
        pass

In [100]:
len(dic_dt['titulo'])
len(dic_dt['link'])

528214

In [101]:
data = pd.DataFrame(dic_dt)

In [102]:
data

,titulo,link
0,Categoria:A Patria Brazileira,https://pt.m.wikisource.org/wiki/Categoria:A_P...
1,Categoria:Arcadismo brasileiro,https://pt.m.wikisource.org/wiki/Categoria:Arc...
2,Categoria:Através do Brasil,https://pt.m.wikisource.org/wiki/Categoria:Atr...
3,Categoria:Barroco brasileiro,https://pt.m.wikisource.org/wiki/Categoria:Bar...
4,Categoria:Contos brasileiros,https://pt.m.wikisource.org/wiki/Categoria:Con...
...,...,...
528209,Thể loại:Sân khấu Brasil — vietnamita,https://vi.wikipedia.org/wiki/Th%E1%BB%83_lo%E...
528210,Editar hiperligações interlínguas,https://www.wikidata.org/wiki/Special:EntityPa...
528211,m:Privacy policy/pt-br,https://meta.wikimedia.org/wiki/Privacy_policy...
528212,Wikipédia:Sobre,https://pt.m.wikisource.org/wiki/Wikip%C3%A9di...


In [105]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///links.db', echo=False)

In [107]:
data['titulo'].to_sql('titulo', con=engine)

In [109]:
data['link'].to_sql('link', con=engine)